In [1]:
import json
import biosig
import matplotlib.pyplot as plt
import numpy as np
from torch import nn
import math
from typing import Literal
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from mne.decoding import CSP
from wave_dataloader import MotorImageryDataset, EVENT_TYPES, Util
import torch
import torch.utils
import torch.utils.data


In [2]:
dataloader = Util.read_data("B0101T.gdf", n_workers=0, bsz=4, item_length=500)
for data, label in dataloader:
    break
print(label)

tensor([32766,   276,   277,  1081])


In [ ]:
device = 'cpu'
fs = 250
channels = 6
num_input = 1
num_class = 14
signal_length = 500
kernel_size_1 = (1, round(fs/2))
kernel_size_2 = (channels, 1)
kernel_size_3 = (1, round(fs/8))
kernel_size_4 = (1, 1)

kernel_avg_pool_1 = (1, 4)
kernel_avg_poll_2 = (1, 8)
dropout_rate = 0.2

ks0 = int(round((kernel_size_1[0]-1)/2))
ks1 = int(round((kernel_size_1[0]-1)/2))
kernel_padding_1 = (ks0, ks1 - 1)
ks0 = int(round((kernel_size_3[0]-1)/2))
ks1 = int(round((kernel_size_3[0]-1)/2))
kernel_padding_3 = (ks0, ks1 - 1)

In [9]:
class EEGNet(nn.Module):
    def __init__(self, 
        channels: int = 6,
        sampling_rate: int = 250,
        dropout_rate: float = 0.5,
        kernel_length: int = 64,
        f1: int = 8,
        D: int = 2,
        f2:int = 16,
        norm_rate: float = 0.25,
        output_size: int = 1
    ):
        super(EEGNet, self).__init__()
        #Norm
        #Block 1
        # self.block1 = nn.Sequential(
        #     nn.BatchNorm2d(f1),
        #     nn.Conv2d(f1, f1 * D, kernel_size=(channels, 1), groups=f1, bias=False),
        #     nn.BatchNorm2d(f1 * D),
        #     nn.ELU(),
        #     nn.AvgPool2d(kernel_size=(1,4)),
        #     nn.Dropout(p=dropout_rate)
        # )
        self.conv1 = nn.Conv2d(
            in_channels=channels,
            out_channels=f1,
            kernel_size=(1, kernel_length),
            padding='same',
            bias=False
        )
        self.bn1 = nn.BatchNorm2d(f1)
        self.conv2 = nn.Conv2d(f1, f1 * D, kernel_size=(channels, 1), groups=f1, bias=False)
        self.bn2 = nn.BatchNorm2d(f1 * D)
        self.act = nn.ELU()
        self.avg1 = nn.AvgPool2d(kernel_size=(1, 4))
        self.dropout = nn.Dropout(p=dropout_rate)
        #Block 2
        self.block2 = nn.Sequential(
            nn.Conv2d(f1 * D, f2, kernel_size=(1, 16), padding='same', bias=False),
            nn.BatchNorm2d(f2),
            nn.ELU(),
            nn.AvgPool2d(kernel_size=(1, 8)),
            nn.Dropout2d(p=dropout_rate)    
        )
        
        #FFW
        self.ffw = nn.Sequential(
            nn.Flatten(),
            nn.Linear(f2 * (sampling_rate // 4) // 8, output_size),
            nn.Softmax(dim=1)
        )
        #Norm
        #elu
        #AvgPool2D
        #dropout
        #SepConv2D
        #Norm
        #Elu
        #AvgPool2d
        #dropout
        #flat
        #dense
        #softmax

    def forward(self, x: torch.Tensor):
        # x = self.block1(x)
        print("I", x.shape)
        x = self.conv1(x)
        print("CON1", x.shape)
        x = self.bn1(x)
        print("BN", x.shape)
        x = self.conv2(x)
        print("CON2", x.shape)
        x = self.bn2(x)
        print("BN", x.shape)
        x = self.act(x)
        print("ACT", x.shape)
        x = self.avg1(x)
        print("AVG", x.shape)
        x = self.dropout(x)
        
        x = self.block2(x)
        x = self.ffw(x)
        return x

In [4]:
#data format [trials, channels, samples, kernels]
kernels = 1
channels = 6
samples = 250

In [5]:
data: torch.Tensor = data.unsqueeze(3)
print(data.shape)

torch.Size([4, 500, 6, 1])


In [6]:
data = data.permute(0,2,1,3)

In [7]:
print(data.shape)
#data format [trials, channels, samples, kernels]

torch.Size([4, 6, 500, 1])


In [12]:
model = EEGNet(
    channels=channels,
    sampling_rate=samples,
    output_size=14,
    kernel_length=32,
    f1=8,
    D=2,
    f2=16,
    dropout_rate=0.5
)
ouput = model.forward(data.float())

I torch.Size([4, 6, 500, 1])
CON1 torch.Size([4, 8, 500, 1])
BN torch.Size([4, 8, 500, 1])
CON2 torch.Size([4, 16, 495, 1])
BN torch.Size([4, 16, 495, 1])
ACT torch.Size([4, 16, 495, 1])


RuntimeError: Given input size: (16x495x1). Calculated output size: (16x495x0). Output size is too small

In [ ]:
mo